In [1]:
# %% General Modules
import numpy as np
import pandas as pd
import os
# %% Special Modules - deep learning
import torch
from torch.utils.data import TensorDataset

# %% Internal Modules
#from utils.file_utils import save_pkl

# %%

# %% [markdown]
## Main

# %% OnI: Instance Level Data
def filter_bag_inst(path, filt):
    #### read csv 
    #path = "inst.csv"
    #print(path)
    data = pd.read_csv(path)
    #print(dataInst.shape)
    
    if filt:
        dict1 = data.groupby('bag')['bag'].agg('count').to_dict()
        l = [k for k, v in dict1.items() if v < 4]
        data1 = data[~data['bag'].isin(l)]
    else:
        data1 = data
        
    return data1

In [2]:
# dict1 = dataInst.groupby('bag')['bag'].agg('count').to_dict()

# [k for k, v in dict1.items() if v < 4]

In [3]:
dataInst = filter_bag_inst("inst.csv", True)

In [4]:
# dataInst

In [5]:
# %% Assign into NumPy Array
insts = np.array( dataInst.iloc[:,0] )
bags = np.array( dataInst.iloc[:,1] )
cases = np.array( dataInst.iloc[:,2] )
labels = np.array( dataInst.iloc[:,3] )
feats = np.array( dataInst.iloc[:,4:] )

# %% PyTorch Tensor
label2bool = np.array([True if l == 'CASE' else False for l in labels])
data = TensorDataset(torch.Tensor(feats), torch.Tensor(label2bool))

In [6]:
dataInst.iloc[:,1:4].head()

,bag,case,label
0,1,1,CTR
1,1,1,CTR
2,1,1,CTR
3,1,1,CTR
4,1,1,CTR


In [7]:
# %% Bag Level Data
dataBag = dataInst.iloc[:,1:4]
print( dataBag.shape )
###
dataBag = dataBag.drop_duplicates()
dataBag = dataBag.rename( columns={
	'bag': 'slide_id',
	'case': 'case_id'
} )
dataBag.to_csv("bag.csv", index = False)

(928, 3)


In [8]:
# %% OnI: Bag Level Data
# path = "{}/bag.csv".format( args.dir_oniput )
# print(path)
# dataBag.to_csv( path, index=False )

# %% List of Bags
bags_unique = np.array(dataBag['slide_id'])
print(bags_unique)

[ 1  2  3  4  5  6  7  8  9 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 43 44 45 46 47 48 49 50 51
 52 53 54 55 56]


In [9]:
# %% Instances in Every Bag
from utils.file_utils import load_pkl, save_pkl
dir = "/Users/chitrakbanerjee/Desktop/omics/tabular-clam/commingle/bags"
print(dir)
os.makedirs( dir, exist_ok=True )
###
insts_in_bags = {}
for b in bags_unique: # this iterates 56 times
    # array of 1:56
    insts_sub = np.where(bags == b)[0]
    # for each of 56 subjects tensor of features
    feats_sub = data.tensors[0][insts_sub,:]
    # insts in bag
    insts_in_bags[b] = insts[insts_sub]
    path = "{}/{}.pt".format( dir, b )
    #print(path)
    torch.save( feats_sub, path )
    #print(insts_sub, feats_sub, insts_in_bags[b])
    #print(insts_sub.shape, feats_sub.shape, insts_in_bags[b].shape)

###### this code is redundant    
# %% Cell Counts
# for ( key, values ) in insts_in_bags.items():
# 	print( str(key) + ':' + str( len(values) ) )

# # %%
path = "/Users/chitrakbanerjee/Desktop/omics/tabular-clam/commingle/insts_in_bags.pkl"
# print(path)
save_pkl( path, insts_in_bags )

# # %%

# # %% [markdown]
# ## EoF

/Users/chitrakbanerjee/Desktop/omics/tabular-clam/commingle/bags


In [10]:
args = pd.DataFrame({'dir_project': '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/commingle/',
        'dir_code': '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/commingle/',
        'results_dir': '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/commingle/result/',
        'dir_oniput': '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/commingle/',
        'dir_split' : '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/commingle/split/',
        'dir_result' : '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/commingle/',
        'cell_barcode_path': '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/commingle/insts_in_bags.pkl',
        'ckpt_path': '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/commingle/30_5/s_1_checkpoint.pt',
        'feature_path': '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/commingle/bags',
        'meta_path': '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/commingle/bag.csv',
        'cv_fold': 5,
        #'cv_val': 0.3,
        'cv_test': 0.3,
        'cv_train': 1,
        'cv_equal': True,
        'opt': 'adam',
        'log_data': True,
        'n_classes': 2,
        'lr': 1e-4,
        'reg': 1e-5, # []
        'drop_out': True,
        'weighted_sample': True,
        'max_epochs': 400,
        'early_stopping': False,
        'model_type': 'clam_mb',
        'bag_loss': 'svm',
        'inst_loss': 'ce',
        'bag_weight': 0.7,
        'B': 2,
        'model_size': 'small',
        'no_inst_cluster': False,
        'subtyping': False,
        'log_data': True,
        'testing': False
       }, index = [0])


In [11]:
import os
os.makedirs( args['dir_split'].item(), exist_ok=True )
print(args['dir_split'].item())
 

# %% General Modules
import numpy as np
import pandas as pd
os.path.join('/Users/chitrakbanerjee/Desktop/omics/tabular-clam/commingle/')
# %% Internal Modules
from datasets.dataset_generic import Generic_WSI_Classification_Dataset
from datasets.dataset_generic import save_splits

# %%

# %% [markdown]
## Preprocessing

# %% Data
# if ( args.job_data == '20210903-1-AT - AD' ):
n_classes = 2
seed =1
cv_fold =5
#cv_val =0.3
cv_test=0.3
cv_train =1
cv_equal = True

dataset = Generic_WSI_Classification_Dataset(csv_path = "bag.csv",
                        label_dict = { "CTR": 0, "CASE": 1},seed = 1, 
                                             patient_strat=True, ignore=[])
# elif ( args.job_data == '20230412-1-HM' ):
# 	args.n_classes = 2
# 	dataset = Generic_WSI_Classification_Dataset(
# 		csv_path = os.path.join( args.dir_oniput, 'bag.csv' ),
# 		label_dict = { 'CTR': 0, args.job_label: 1 },
# 		seed=args.seed, patient_strat=True, ignore=[]
# 	)
# else:
# 	raise NotImplementedError

# %%

# %% [markdown]
## Main


# %% Preparation
num_slides_cls = np.array([len(cls_ids) for cls_ids in dataset.patient_cls_ids])
#val_num = np.round( num_slides_cls * cv_val ).astype(int)
test_num = np.round( num_slides_cls * cv_test ).astype(int)
###
if cv_equal is True:
#    val_num = [ min(val_num) for n in val_num ]
    test_num = [ min(test_num) for n in test_num ]
###
dataset.create_splits(
    k = cv_fold,
    test_num = test_num,
    label_frac = cv_train)

# %% Run: Split
for f in range(cv_fold):
    dataset.set_splits()
    descriptor_df = dataset.test_split_gen( return_descriptor=True )
    splits = dataset.return_splits( from_id=True )
    filename = '{}_{}_{}'.format(
        int( 100 * cv_test ), cv_fold, f
    )
    save_splits( splits, ['train', 'test'], os.path.join(
        'split', filename + '.csv') )
    save_splits( splits, ['train', 'test'], os.path.join(
        'split', filename + '_bool' + '.csv'
    ), boolean_style=True )
    descriptor_df.to_csv( os.path.join(
        'split', filename + '_descriptor' + '.csv'
    ) )

# %%

# %% [markdown]
## EoF


/Users/chitrakbanerjee/Desktop/omics/tabular-clam/commingle/split/
label column: label
label dictionary: {'CTR': 0, 'CASE': 1}
number of classes: 2
slide-level counts:  
 0    37
1    16
Name: label, dtype: int64
Patient-LVL; Number of samples registered in class 0: 37
Slide-LVL; Number of samples registered in class 0: 37
Patient-LVL; Number of samples registered in class 1: 16
Slide-LVL; Number of samples registered in class 1: 16

number of training samples: 43
number of samples in cls 0: 32
number of samples in cls 1: 11

number of test samples: 10
number of samples in cls 0: 5
number of samples in cls 1: 5



number of training samples: 43
number of samples in cls 0: 32
number of samples in cls 1: 11

number of test samples: 10
number of samples in cls 0: 5
number of samples in cls 1: 5



number of training samples: 43
number of samples in cls 0: 32
number of samples in cls 1: 11

number of test samples: 10
number of samples in cls 0: 5
number of samples in cls 1: 5



number of 

In [12]:
# %% General Modules
import itertools
import math
import pdb
import random
###
import numpy as np
import pandas as pd

# %% Special Modules - deep learning
import torch
#from torch.utils.data import DataLoader, sampler
#import torch.nn as nn
#import torch.nn.functional as F
###
device = torch.device( "cuda" if torch.cuda.is_available() else "cpu" )

# %% Internal Modules
from datasets.dataset_generic import Generic_MIL_Dataset
from utils.core_utils import train
from utils.file_utils import load_pkl, save_pkl
from utils.utils import *

# %%

# %% [markdown]
## Preprocessing

# %% Data

dataset = Generic_MIL_Dataset(
		data_dir = '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/commingle/',
		csv_path = os.path.join( '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/commingle/', 'bag.csv' ),
		label_dict = { 'CTR': 0, "CASE": 1 },
		seed=1, patient_strat=True, ignore=[]
	)

# %%

# %% [markdown]
## Functions

# %% Seeding
def seed_torch(seed):
	random.seed(seed)
	os.environ['PYTHONHASHSEED'] = str(seed)
	np.random.seed(seed)
	torch.manual_seed(seed)
	if device.type == 'cuda':
		torch.cuda.manual_seed(seed)
		torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
	torch.backends.cudnn.benchmark = False
	torch.backends.cudnn.deterministic = True

# %% Single Run of CLAM-SMILE
def runSingle( f, args ):
	( train_dataset, test_dataset ) = dataset.return_splits(
		from_id = False, 
		csv_path = os.path.join( '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/commingle/split/', '{}_{}_{}'.format(
			 int( 100 * cv_test ), cv_fold, f
		) + '.csv' ),
		 test_frac=args['cv_test'].item()
	)
	datasets = ( train_dataset, test_dataset )
	return train( datasets, f, args )

# %%

# %% [markdown]
## Main

# %% Main
if __name__ == "__main__":
    args.results_dir = '{}/{}_{}'.format( args['dir_result'].item(),
         int( 100 * args['cv_test'].item() ),
                                            args['cv_fold'].item())
    os.makedirs( args['results_dir'].item(), exist_ok=True )
    ###
    folds = np.arange( 0, args['cv_fold'].item())

    all_stopping_epoch = []
    all_train_auc = []
    all_train_acc = []
    all_test_auc = []
    all_test_acc = []
    ###
    all_train_acc_vec = []
    all_train_auc_vec = []
    ###
    for f in folds:
        (results, train_auc, train_acc, test_auc, test_acc,
         stopping_epoch, train_acc_vec, train_auc_vec) = runSingle( f, args )
        all_stopping_epoch.append(stopping_epoch)
        all_train_auc.append(train_auc)
        all_train_acc.append(train_acc)
        if cv_test > 0:
            all_test_auc.append(test_auc)
            all_test_acc.append(test_acc)
        else:
            all_test_auc.append(-1)
            all_test_acc.append(-1)
        all_train_acc_vec.append( train_acc_vec )
        all_train_auc_vec.append( train_auc_vec )
        ###
        results_dir = args['results_dir'].item()
        path = '{}/{}.pkl'.format( results_dir, f )
        save_pkl( path, results )
        ###
        conv = pd.DataFrame( {'train_acc': train_acc_vec, 'train_auc': train_auc_vec} )
        path = '{}/{}.csv'.format( results_dir, f )
        conv.to_csv(path)
    ###
    final_df = pd.DataFrame({'folds': folds, 'stopping_epoch': all_stopping_epoch,
        'train_auc': all_train_auc, 'train_acc' : all_train_acc,
                             'test_auc': all_test_auc, 'test_acc' : all_test_acc} )
    ###
    final_df = final_df.append( { 'folds': '', 'stopping_epoch': np.mean(all_stopping_epoch),
        'train_auc': np.mean(all_train_auc),
        'train_acc' : np.mean(all_train_acc),
        'test_auc': np.mean(all_test_auc),
        'test_acc': np.mean(all_test_acc)
    }, ignore_index = True )
    ###
    print(final_df)
    path = '{}/summary.csv'.format( results_dir )
    final_df.to_csv(path)
    ###
    print("finished!")
print("end script")

# %%

# %% [markdown]
## EoF


label column: label
label dictionary: {'CTR': 0, 'CASE': 1}
number of classes: 2
slide-level counts:  
 0    37
1    16
Name: label, dtype: int64
Patient-LVL; Number of samples registered in class 0: 37
Slide-LVL; Number of samples registered in class 0: 37
Patient-LVL; Number of samples registered in class 1: 16
Slide-LVL; Number of samples registered in class 1: 16

Training Fold 0!

Init train/val/test splits... 
Done!
Training on 43 samples
Testing on 10 samples

Init loss function... Setting tau to 1.0
Done!

Init Model... Done!
CLAM_MB(
  (attention_net): Sequential(
    (0): Linear(in_features=315, out_features=100, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
    (3): Attn_Net_Gated(
      (attention_a): Sequential(
        (0): Linear(in_features=100, out_features=50, bias=True)
        (1): Tanh()
        (2): Dropout(p=0.25, inplace=False)
      )
      (attention_b): Sequential(
        (0): Linear(in_features=100, out_features=50, bias=True)
        (


Test Set, test_loss: 1548.7992, test_error: 0.6000, auc: 0.4200
class 0 clustering acc 0.8: correct 16/20
class 1 clustering acc 0.2: correct 4/20
class 0: acc 0.2, correct 1/5
class 1: acc 0.6, correct 3/5
epoch:10


batch 19, loss: 0.0000, instance_loss: 5088.8022, weighted_loss: 1526.6407, label: 1, bag_size: 18
batch 39, loss: 12468.8965, instance_loss: 10911.6396, weighted_loss: 12001.7197, label: 0, bag_size: 15


class 0 clustering acc 0.46511627906976744: correct 40/86
class 1 clustering acc 0.45348837209302323: correct 39/86
Epoch: 10, train_loss: 4615.0228, train_clustering_loss:  16491.6502, train_error: 0.4884
class 0: acc 0.391304347826087, correct 9/23
class 1: acc 0.65, correct 13/20

Test Set, test_loss: 1657.4128, test_error: 0.6000, auc: 0.4000
class 0 clustering acc 0.7: correct 14/20
class 1 clustering acc 0.3: correct 6/20
class 0: acc 0.2, correct 1/5
class 1: acc 0.6, correct 3/5
epoch:11


batch 19, loss: 0.0000, instance_loss: 0.0000, weighted_loss: 0.0000, la

epoch:22


batch 19, loss: 0.0000, instance_loss: 2172.7849, weighted_loss: 651.8355, label: 0, bag_size: 9
batch 39, loss: 0.0000, instance_loss: 4699.9062, weighted_loss: 1409.9719, label: 1, bag_size: 17


class 0 clustering acc 0.5348837209302325: correct 46/86
class 1 clustering acc 0.5: correct 43/86
Epoch: 22, train_loss: 3292.6878, train_clustering_loss:  13083.3928, train_error: 0.4186
class 0: acc 0.7272727272727273, correct 16/22
class 1: acc 0.42857142857142855, correct 9/21

Test Set, test_loss: 2647.5942, test_error: 0.5000, auc: 0.5000
class 0 clustering acc 0.8: correct 16/20
class 1 clustering acc 0.5: correct 10/20
class 0: acc 0.8, correct 4/5
class 1: acc 0.2, correct 1/5
epoch:23


batch 19, loss: 8381.2480, instance_loss: 6464.2432, weighted_loss: 7806.1465, label: 0, bag_size: 7
batch 39, loss: 0.0000, instance_loss: 6720.7559, weighted_loss: 2016.2268, label: 0, bag_size: 42


class 0 clustering acc 0.5465116279069767: correct 47/86
class 1 clustering acc 0.5930

epoch:34


batch 19, loss: 0.0000, instance_loss: 0.0000, weighted_loss: 0.0000, label: 1, bag_size: 34
batch 39, loss: 0.0000, instance_loss: 5138.2432, weighted_loss: 1541.4730, label: 1, bag_size: 22


class 0 clustering acc 0.5581395348837209: correct 48/86
class 1 clustering acc 0.5: correct 43/86
Epoch: 34, train_loss: 3754.0553, train_clustering_loss:  13630.4656, train_error: 0.3488
class 0: acc 0.6666666666666666, correct 14/21
class 1: acc 0.6363636363636364, correct 14/22

Test Set, test_loss: 4156.2876, test_error: 0.5000, auc: 0.5000
class 0 clustering acc 0.7: correct 14/20
class 1 clustering acc 0.75: correct 15/20
class 0: acc 0.8, correct 4/5
class 1: acc 0.2, correct 1/5
epoch:35


batch 19, loss: 0.0000, instance_loss: 738.0527, weighted_loss: 221.4158, label: 0, bag_size: 14
batch 39, loss: 8043.2144, instance_loss: 15779.2207, weighted_loss: 10364.0166, label: 0, bag_size: 45


class 0 clustering acc 0.43023255813953487: correct 37/86
class 1 clustering acc 0.46511

batch 19, loss: 0.0000, instance_loss: 150.1887, weighted_loss: 45.0566, label: 0, bag_size: 21
batch 39, loss: 0.0000, instance_loss: 21825.0996, weighted_loss: 6547.5303, label: 1, bag_size: 15


class 0 clustering acc 0.5465116279069767: correct 47/86
class 1 clustering acc 0.6395348837209303: correct 55/86
Epoch: 46, train_loss: 3267.9989, train_clustering_loss:  11152.4495, train_error: 0.3488
class 0: acc 0.5238095238095238, correct 11/21
class 1: acc 0.7727272727272727, correct 17/22

Test Set, test_loss: 3432.0537, test_error: 0.6000, auc: 0.4000
class 0 clustering acc 0.6: correct 12/20
class 1 clustering acc 0.8: correct 16/20
class 0: acc 0.6, correct 3/5
class 1: acc 0.2, correct 1/5
epoch:47


batch 19, loss: 0.0000, instance_loss: 6490.6206, weighted_loss: 1947.1863, label: 1, bag_size: 18
batch 39, loss: 0.0000, instance_loss: 5754.0044, weighted_loss: 1726.2014, label: 0, bag_size: 14


class 0 clustering acc 0.4186046511627907: correct 36/86
class 1 clustering acc 0.51

...

batch 19, loss: 0.0000, instance_loss: 2513.1199, weighted_loss: 753.9360, label: 1, bag_size: 22
batch 39, loss: 0.0000, instance_loss: 2.6508, weighted_loss: 0.7952, label: 1, bag_size: 23


class 0 clustering acc 0.5581395348837209: correct 48/86
class 1 clustering acc 0.5581395348837209: correct 48/86
Epoch: 358, train_loss: 1780.3723, train_clustering_loss:  3379.5464, train_error: 0.2558
class 0: acc 0.6666666666666666, correct 14/21
class 1: acc 0.8181818181818182, correct 18/22

Test Set, test_loss: 5196.5673, test_error: 0.5000, auc: 0.5000
class 0 clustering acc 1.0: correct 20/20
class 1 clustering acc 0.85: correct 17/20
class 0: acc 0.8, correct 4/5
class 1: acc 0.2, correct 1/5
epoch:359


batch 19, loss: 0.0000, instance_loss: 3640.8359, weighted_loss: 1092.2509, label: 0, bag_size: 42
batch 39, loss: 0.0000, instance_loss: 7539.4238, weighted_loss: 2261.8271, label: 1, bag_size: 15


class 0 clustering acc 0.7325581395348837: correct 63/86
class 1 clustering acc 0.65116

batch 19, loss: 0.0000, instance_loss: 16.2236, weighted_loss: 4.8671, label: 0, bag_size: 17
batch 39, loss: 2293.4160, instance_loss: 3555.6123, weighted_loss: 2672.0750, label: 1, bag_size: 20


class 0 clustering acc 0.6976744186046512: correct 60/86
class 1 clustering acc 0.5465116279069767: correct 47/86
Epoch: 370, train_loss: 1569.6253, train_clustering_loss:  3000.4930, train_error: 0.2791
class 0: acc 0.8, correct 20/25
class 1: acc 0.6111111111111112, correct 11/18

Test Set, test_loss: 5828.5949, test_error: 0.5000, auc: 0.5000
class 0 clustering acc 0.95: correct 19/20
class 1 clustering acc 0.8: correct 16/20
class 0: acc 0.8, correct 4/5
class 1: acc 0.2, correct 1/5
epoch:371


batch 19, loss: 0.0000, instance_loss: 0.0000, weighted_loss: 0.0000, label: 1, bag_size: 22
batch 39, loss: 0.0000, instance_loss: 189.3730, weighted_loss: 56.8119, label: 0, bag_size: 16


class 0 clustering acc 0.6744186046511628: correct 58/86
class 1 clustering acc 0.5348837209302325: correc

batch 19, loss: 0.0000, instance_loss: 7578.3071, weighted_loss: 2273.4922, label: 1, bag_size: 14
batch 39, loss: 0.0000, instance_loss: 2992.2759, weighted_loss: 897.6828, label: 1, bag_size: 15


class 0 clustering acc 0.686046511627907: correct 59/86
class 1 clustering acc 0.5581395348837209: correct 48/86
Epoch: 382, train_loss: 973.3042, train_clustering_loss:  2502.8953, train_error: 0.2558
class 0: acc 0.75, correct 15/20
class 1: acc 0.7391304347826086, correct 17/23

Test Set, test_loss: 4581.6068, test_error: 0.5000, auc: 0.5000
class 0 clustering acc 0.9: correct 18/20
class 1 clustering acc 0.75: correct 15/20
class 0: acc 0.8, correct 4/5
class 1: acc 0.2, correct 1/5
epoch:383


batch 19, loss: 0.0000, instance_loss: 0.0000, weighted_loss: 0.0000, label: 0, bag_size: 45
batch 39, loss: 0.0000, instance_loss: 0.0000, weighted_loss: 0.0000, label: 1, bag_size: 17


class 0 clustering acc 0.6744186046511628: correct 58/86
class 1 clustering acc 0.7209302325581395: correct 6

batch 19, loss: 0.0000, instance_loss: 81.2100, weighted_loss: 24.3630, label: 0, bag_size: 20
batch 39, loss: 0.0000, instance_loss: 752.8364, weighted_loss: 225.8509, label: 0, bag_size: 24


class 0 clustering acc 0.5813953488372093: correct 50/86
class 1 clustering acc 0.6046511627906976: correct 52/86
Epoch: 394, train_loss: 1711.0740, train_clustering_loss:  1831.6646, train_error: 0.1860
class 0: acc 0.8461538461538461, correct 22/26
class 1: acc 0.7647058823529411, correct 13/17

Test Set, test_loss: 5133.1457, test_error: 0.6000, auc: 0.4000
class 0 clustering acc 0.75: correct 15/20
class 1 clustering acc 0.75: correct 15/20
class 0: acc 0.6, correct 3/5
class 1: acc 0.2, correct 1/5
epoch:395


batch 19, loss: 0.0000, instance_loss: 152.5352, weighted_loss: 45.7605, label: 1, bag_size: 15
batch 39, loss: 6506.7109, instance_loss: 1145.7979, weighted_loss: 4898.4370, label: 0, bag_size: 45


class 0 clustering acc 0.5813953488372093: correct 50/86
class 1 clustering acc 0.593

/var/folders/45/fswzhp4j1qx7m6_9ty17lp500000gn/T/ipykernel_63862/3127776541.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append( { 'folds': '', 'stopping_epoch': np.mean(all_stopping_epoch),


In [13]:
# %% [markdown]
## Command Line Interfaces

# %% Creating a Parser
# import argparse
# parser = argparse.ArgumentParser()

# # %% Adding Arguments
# dir_project = '/SFS/user/ry/ohjihw/project/015'
# parser.add_argument( '--dir_project', type=str, default=dir_project )
# parser.add_argument( '--dir_code', type=str, default='code/python' )
# parser.add_argument( '--dir_oniput', type=str, default='oniput' )
# parser.add_argument( '--dir_split', type=str, default='split' )
# parser.add_argument( '--dir_result', type=str, default='result' )
# ###
# parser.add_argument( '--job_data', type=str, default='20230412-1-HM' )
# parser.add_argument( '--job_task', type=str, default='Glomeruli' )
# parser.add_argument( '--job_task', type=str, default='Tubules' )
# parser.add_argument( '--job_label', type=str, default='CASE' )
# ###
# parser.add_argument( '--cv_fold', type=int, default=1 )
# parser.add_argument( '--cv_val', type=float, default=0.0 )
# parser.add_argument( '--cv_test', type=float, default=0.0 )
# parser.add_argument( '--cv_train', type=float, default=1.0 )
# parser.add_argument( '--cv_equal', type=bool, default=True )
# ###
# parser.add_argument( '--opt', type=str, choices=[ 'adam', 'sgd' ], default='adam' )
# parser.add_argument( '--lr', type=float, default=1e-4, \
# 	help='learning rate (default: 1e-4)' )
# parser.add_argument( '--reg', type=float, default=1e-5, \
# 	help='weight decay (default: 1e-5)' )
# parser.add_argument( '--drop_out', action='store_true', default=True, \
# 	help='enabel dropout (p=0.25)' )
# parser.add_argument( '--weighted_sample', action='store_true', default=True, \
# 	help='enable weighted sampling' )
# parser.add_argument( '--max_epochs', type=int, default=400, \
# 	help='maximum number of epochs to train (default: 200)' )
# parser.add_argument( '--early_stopping', action='store_true', default=False, \
# 	help='enable early stopping' )
# ### CLAM
# parser.add_argument( '--model_type', type=str, choices=[ 'clam_sb', 'clam_mb', 'mil' ], default='clam_sb', \
# 	help='type of model (default: clam_sb, clam w/ single attention branch)' )
# parser.add_argument( '--bag_loss', type=str, choices=[ 'svm', 'ce' ], default='ce', \
# 	help='sample-level classification loss function (default: ce)' )
# parser.add_argument( '--inst_loss', type=str, choices=[ 'svm', 'ce', None ], default='svm', \
# 	help='instance-level clustering loss function (default: None)' )
# parser.add_argument( '--bag_weight', type=float, default=0.7, \
# 	help='clam: weight coefficient for bag-level loss (default: 0.7)' )
# parser.add_argument( '--B', type=int, default=10, \
# 	help='numbr of positive/negative patches to sample for clam' )
# parser.add_argument( '--model_size', type=str, choices=[ 'small', 'big' ], default='small', \
# 	help='size of model, does not affect mil' )
# parser.add_argument( '--no_inst_cluster', action='store_true', default=False, \
# 	help='disable instance-level clustering' )
# parser.add_argument( '--subtyping', action='store_true', default=False, \
# 	help='subtyping problem' )
# 	# In case you want to modify, maybe try first with 'bag_weight' and 'B'. - JO
# ### Miscellaneous
# parser.add_argument( '--log_data', action='store_true', default=True, \
# 	help='log data using tensorboard' )
# parser.add_argument( '--testing', action='store_true', default=False, \
# 	help='debugging tool' )

# %% Parsing Arguments
# in_ipython = hasattr(__builtins__,'__IPYTHON__')
# args = parser.parse_args("") if in_ipython else parser.parse_args()
# for ( key, value ) in vars(args).items():
# 	print( 'args' + '.' + key + ':', value )

# %%

# %% [markdown]
## Header

# %% Working Directory
# import os
# args.dir_code = os.path.join( args.dir_project, args.dir_code )
# print( 'args.dir_code:', args.dir_code )
# os.chdir( args.dir_code )
# args.dir_oniput = os.path.join( args.dir_project, args.dir_oniput )
# args.dir_oniput = os.path.join( args.dir_oniput, args.job_data )
# args.dir_oniput = os.path.join( args.dir_oniput, args.job_task )
# print( 'args.dir_ioput:', args.dir_oniput )
# args.dir_split = os.path.join( args.dir_oniput, args.dir_split )
# os.makedirs( args.dir_split, exist_ok=True )
# args.dir_result = os.path.join( args.dir_oniput, args.dir_result )
# os.makedirs( args.dir_result, exist_ok=True )

# %% General Modules
# import itertools
# import math
# import pdb
# import random
# ###
# import matplotlib.pylab as plt
# import numpy as np
# import pandas as pd
# import seaborn as sns

# # %% Special Modules - deep learning
# import torch
# import torch.nn.functional as F
# ###
# device = torch.device( "cuda" if torch.cuda.is_available() else "cpu" )

# # %% Internal Modules
from models.model_clam import CLAM_MB, CLAM_SB
from utils.utils import *
from utils.file_utils import save_pkl, load_pkl

# %%

# %% [markdown]
## Preparation

# %%
def initiate_model(model_args, ckpt_path):
    print('Init Model')    
    model_dict = {
        "dropout": model_args['drop_out'].item(),
        'n_classes': model_args['n_classes'].item(), # model_args.n_classes,
        "k_sample": model_args['B'].item()
    }
    if model_args['model_size'].item() is not None and model_args['model_type'].item() in ['clam_sb', 'clam_mb']:
        model_dict.update({"size_arg": model_args['model_size'].item()})
    if model_args['model_type'].item() =='clam_sb':
        model = CLAM_SB(**model_dict)
    elif model_args['model_type'].item() =='clam_mb':
        model = CLAM_MB(**model_dict)
    else: # args.model_type == 'mil'
        if model_args['n_classes'].item() > 2:
            model = MIL_fc_mc(**model_dict)
        else:
            model = MIL_fc(**model_dict)
    print_network(model)
    ckpt = torch.load(ckpt_path)
    #print_network(ckpt)
    ckpt_clean = {}
    for key in ckpt.keys():
        if 'instance_loss_fn' in key:
            continue
        ckpt_clean.update({key.replace('.module', ''):ckpt[key]})
    model.load_state_dict(ckpt_clean, strict=True)
    model.relocate()
    model.eval()
    return model

# %%
def infer_single_slide(model, features, label, reverse_label_dict, k=1, instance_eval = False, label_tensor = None):
    features = features.to(device)
    with torch.no_grad():
        if isinstance(model, (CLAM_SB,)):
            #model_results_dict = model(features)
            logits, Y_prob, Y_hat, A, results_dict = model(features, instance_eval = instance_eval, label = label_tensor)
            print(results_dict)
            Y_hat = Y_hat.item()
            if isinstance(model, (CLAM_MB,)):
                A = A[Y_hat]
            A = A.view(-1, 1).cpu().numpy()
        else:
            raise NotImplementedError
        print('Y_hat: {}, Y: {}, Y_prob: {}'.format(reverse_label_dict[Y_hat], label, ["{:.4f}".format(p) for p in Y_prob.cpu().flatten()]))
        probs, ids = torch.topk(Y_prob, k)
        probs = probs[-1].cpu().numpy()
        ids = ids[-1].cpu().numpy()
        preds_str = np.array([reverse_label_dict[idx] for idx in ids])
    return ids, preds_str, probs, A, results_dict


# %%

# %% [markdown]
## Line by Line

# %%
def extract_attention_scores(args, cell_barcode_path, ckpt_path, feature_path, meta_path):
    ###
    cell_barcode_dict = load_pkl(cell_barcode_path)
    metadata = pd.read_csv(meta_path)
    files = os.listdir(feature_path)
    cell_weights = torch.Tensor()
    ###
    sample_list = []
    barcode_list = []
    results_dict_list = []
    ###
    for f in files:
        features = torch.load(os.path.join(feature_path, f))
        model = initiate_model(args, ckpt_path)
        sample_id = f.split('.')[0]
        if sample_id not in str( cell_barcode_dict.keys() ):
            continue
        cell_barcodes = cell_barcode_dict[ int( sample_id )]
        label = metadata[ metadata['slide_id'] == int( sample_id )].label.values[0]
        label_dict = {'CTR':0, 'CASE':1 }
        class_labels = list(label_dict.keys())
        class_encodings = list(label_dict.values())
        label_tensor = torch.tensor([label_dict[label]])
        reverse_label_dict = {class_encodings[i]: class_labels[i] for i in range(len(class_labels))} 
        ids, preds_str, probs, A, results_dict  = infer_single_slide(model, features, label, reverse_label_dict, k=1,
                                                        instance_eval = True, label_tensor= label_tensor)
        weights = F.softmax(torch.Tensor(A), dim = 0).view(1, -1)
        cell_weights = torch.cat((cell_weights, weights), dim = 1)
        sample_list.extend(np.repeat(sample_id, features.shape[0]))
        barcode_list.extend(cell_barcodes)
        results_dict_list.append(results_dict)
    return cell_weights, sample_list, barcode_list, results_dict_list

# %% [markdown]
## Main

# %%
# args.results_dir = '{}/{}_{}_{}'.format(
# 	args.dir_result,
# 	int( 100 * args.cv_val ), int( 100 * args.cv_test ), args.cv_fold
# )
###
# cell_barcode_path = "{}/insts_in_bags.pkl".format( args.dir_oniput )
# ckpt_path = "{}/s_0_checkpoint.pt".format( args.results_dir )
# feature_path = "{}/bags".format( args.dir_oniput )
# meta_path = "{}/bag.csv".format( args.dir_oniput )

cell_barcode_path = args['cell_barcode_path'].item()
ckpt_path = args['ckpt_path'].item()
feature_path = args['feature_path'].item()
meta_path = args['meta_path'].item()

#%%
# args.ckpt_path = ckpt_path # os.path.join(ckpt_path, f, 's_0_checkpoint.pt')
# args.k_sample = args.B #int(f.split('_B_')[1].split('_bag')[0])
cell_weights, sample_list, barcode_list, results_dict_list = extract_attention_scores( args,
                                                                        cell_barcode_path, #args.barcode_path,
                                                                        ckpt_path, #args.ckpt_path,
                                                                        feature_path, #args.feature_path, 
                                                                        meta_path) #args.meta_path 

# %%
DF = pd.DataFrame( {'bag': sample_list, 'inst': barcode_list, 'weight': cell_weights[0,:]} )
DF

# %%
path = "/Users/chitrakbanerjee/Desktop/omics/tabular-clam/commingle/weights.csv"
DF.to_csv(path)

# %%

# %% [markdown]
## EoF


Init Model
CLAM_MB(
  (attention_net): Sequential(
    (0): Linear(in_features=315, out_features=100, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
    (3): Attn_Net_Gated(
      (attention_a): Sequential(
        (0): Linear(in_features=100, out_features=50, bias=True)
        (1): Tanh()
        (2): Dropout(p=0.25, inplace=False)
      )
      (attention_b): Sequential(
        (0): Linear(in_features=100, out_features=50, bias=True)
        (1): Sigmoid()
        (2): Dropout(p=0.25, inplace=False)
      )
      (attention_c): Linear(in_features=50, out_features=2, bias=True)
    )
  )
  (classifiers): ModuleList(
    (0-1): 2 x Linear(in_features=100, out_features=1, bias=True)
  )
  (instance_classifiers): ModuleList(
    (0-1): 2 x Linear(in_features=100, out_features=2, bias=True)
  )
  (instance_loss_fn): CrossEntropyLoss()
)
Total number of parameters: 42408
Total number of trainable parameters: 42408
{'instance_loss': tensor(1200.1807), 'inst_labels': ar

CLAM_MB(
  (attention_net): Sequential(
    (0): Linear(in_features=315, out_features=100, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
    (3): Attn_Net_Gated(
      (attention_a): Sequential(
        (0): Linear(in_features=100, out_features=50, bias=True)
        (1): Tanh()
        (2): Dropout(p=0.25, inplace=False)
      )
      (attention_b): Sequential(
        (0): Linear(in_features=100, out_features=50, bias=True)
        (1): Sigmoid()
        (2): Dropout(p=0.25, inplace=False)
      )
      (attention_c): Linear(in_features=50, out_features=2, bias=True)
    )
  )
  (classifiers): ModuleList(
    (0-1): 2 x Linear(in_features=100, out_features=1, bias=True)
  )
  (instance_classifiers): ModuleList(
    (0-1): 2 x Linear(in_features=100, out_features=2, bias=True)
  )
  (instance_loss_fn): CrossEntropyLoss()
)
Total number of parameters: 42408
Total number of trainable parameters: 42408
{'instance_loss': tensor(913.8657), 'inst_labels': array([1, 1, 0

{'instance_loss': tensor(1812.9169), 'inst_labels': array([1, 1, 0, 0]), 'inst_preds': array([0, 0, 1, 1])}
Y_hat: CTR, Y: CTR, Y_prob: ['1.0000', '0.0000']
Init Model
CLAM_MB(
  (attention_net): Sequential(
    (0): Linear(in_features=315, out_features=100, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
    (3): Attn_Net_Gated(
      (attention_a): Sequential(
        (0): Linear(in_features=100, out_features=50, bias=True)
        (1): Tanh()
        (2): Dropout(p=0.25, inplace=False)
      )
      (attention_b): Sequential(
        (0): Linear(in_features=100, out_features=50, bias=True)
        (1): Sigmoid()
        (2): Dropout(p=0.25, inplace=False)
      )
      (attention_c): Linear(in_features=50, out_features=2, bias=True)
    )
  )
  (classifiers): ModuleList(
    (0-1): 2 x Linear(in_features=100, out_features=1, bias=True)
  )
  (instance_classifiers): ModuleList(
    (0-1): 2 x Linear(in_features=100, out_features=2, bias=True)
  )
  (instance_loss_f

## Validation on unlabeled dataset manipulation and clean up

In [15]:
oob = pd.read_excel('validation_Glomeruli_combined.xlsx', sheet_name='in')

In [16]:
#oob.head()
oob1 = oob.rename({'Var1': 'bag', 'Var3': 'inst'}, axis='columns')

In [17]:
oob1.head()

,bag,Var2,inst,feat1,feat2,feat3,feat4,feat5,feat6,feat7,...,feat306,feat307,feat308,feat309,feat310,feat311,feat312,feat313,feat314,feat315
0,1,1,V10_S_4_0.jpeg,0.79268,0.21862,0.084256,423850,2649.1,515.5,0.32402,...,129.910,192.750,244.990,281.920,312.910,338.62,361.15,381.45,400.68,421.15
1,1,2,V10_S_4_1.jpeg,0.80162,0.19187,0.086713,459760,2688.7,387.0,0.35926,...,115.590,175.060,227.430,265.220,298.050,325.95,348.84,368.02,387.03,406.84
2,1,3,V10_S_4_2.jpeg,0.81578,0.17478,0.074669,450370,2530.2,429.5,0.31225,...,124.220,180.420,231.190,266.180,295.550,323.42,347.42,368.17,387.23,407.42
3,1,4,V10_S_4_0.jpeg,0.79872,0.26091,0.002771,66788,16697.0,420.5,0.25032,...,43.322,62.301,76.722,88.781,99.322,108.84,117.75,126.32,135.32,154.32
4,1,5,V10_S_4_1.jpeg,0.75766,0.30022,0.003555,73692,10527.0,195.0,0.24646,...,46.740,65.744,80.744,94.577,106.500,117.42,127.70,137.29,148.16,168.74


In [18]:
oob1 = oob1.drop('Var2', axis = 1)

In [19]:
oob1['case'] = oob1['bag']

In [20]:
oob1.to_csv('oob_inst.csv', index = False)

## OOB validation

In [14]:
data_oob = filter_bag_inst("oob_inst.csv", True)

In [15]:
data_oob.head(3)

,bag,inst,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,...,feat307,feat308,feat309,feat310,feat311,feat312,feat313,feat314,feat315,case
0,1,V10_S_4_0.jpeg,0.79268,0.21862,0.084256,423850,2649.1,515.5,0.32402,0.76100,...,192.75,244.99,281.92,312.91,338.62,361.15,381.45,400.68,421.15,1
1,1,V10_S_4_1.jpeg,0.80162,0.19187,0.086713,459760,2688.7,387.0,0.35926,0.75088,...,175.06,227.43,265.22,298.05,325.95,348.84,368.02,387.03,406.84,1
2,1,V10_S_4_2.jpeg,0.81578,0.17478,0.074669,450370,2530.2,429.5,0.31225,0.76858,...,180.42,231.19,266.18,295.55,323.42,347.42,368.17,387.23,407.42,1


In [16]:
# %% Assign into NumPy Array
insts_oob = np.array( data_oob['inst'] )
bags_oob = np.array( data_oob['bag'] )
# cases = np.array( dataInst.iloc[:,2] )
# labels = np.array( dataInst.iloc[:,3] )
feats_oob = np.array( data_oob.loc[:, data_oob.columns.str.startswith('feat')] )

# %% PyTorch Tensor
# label2bool = np.array([True if l == 'CASE' else False for l in labels])
data_t_oob = TensorDataset(torch.Tensor(feats_oob)) #, torch.Tensor(label2bool))

In [17]:
# %% Bag Level Data
dataBag_oob = data_oob[['bag', 'case']]
print( dataBag_oob.shape )
###
dataBag_oob = dataBag_oob.drop_duplicates()
dataBag_oob = dataBag_oob.rename( columns={'bag': 'slide_id', 'case': 'case_id'} )
dataBag_oob.to_csv("bag_oob.csv", index = False)

(414, 2)


In [18]:
# %% OnI: Bag Level Data
# path = "{}/bag.csv".format( args.dir_oniput )
# print(path)
# dataBag.to_csv( path, index=False )

# %% List of Bags
bags_unique_oob = np.array(dataBag_oob['slide_id'])
print(bags_unique_oob)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30]


In [19]:
# %% Instances in Every Bag
dir = "{}/bags_oob".format('/Users/chitrakbanerjee/Desktop/omics/tabular-clam/Commingle')
print(dir)
os.makedirs( dir, exist_ok=True )
###
insts_in_bags_oob = {}
for b in bags_unique_oob: # this iterates 56 times
    # array of 1:56
    insts_sub = np.where(bags_oob == b)[0]
    # for each of 56 subjects tensor of features
    feats_sub = data_t_oob.tensors[0][insts_sub,:]
    # insts in bag
    insts_in_bags_oob[b] = insts_oob[insts_sub]
    path = "{}/{}.pt".format( dir, b )
    #print(path)
    torch.save( feats_sub, path )
    #print(insts_sub, feats_sub, insts_in_bags_oob[b])
    #print(insts_sub.shape, feats_sub.shape, insts_in_bags[b].shape)

###### this code is redundant    
# %% Cell Counts
# for ( key, values ) in insts_in_bags.items():
# 	print( str(key) + ':' + str( len(values) ) )

# # %%
path = "/Users/chitrakbanerjee/Desktop/omics/tabular-clam/Commingle/insts_in_bags_oob.pkl"
# print(path)
save_pkl( path, insts_in_bags_oob )

# # %%

# # %% [markdown]
# ## EoF

/Users/chitrakbanerjee/Desktop/omics/tabular-clam/Commingle/bags_oob


In [20]:
args = pd.DataFrame({'dir_project': '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/Commingle/',
        'dir_code': '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/Commingle/',
        'results_dir': '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/Commingle/result/',
        'dir_oniput': '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/Commingle/',
        'dir_split' : '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/Commingle/split/',
        'dir_result' : '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/Commingle/result/',
        'cell_barcode_path': '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/Commingle/insts_in_bags_oob.pkl',
        'ckpt_path': '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/Commingle/30_5/s_1_checkpoint.pt',
        'feature_path': '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/Commingle/bags_oob',
        'meta_path': '/Users/chitrakbanerjee/Desktop/omics/tabular-clam/Commingle/bag_oob.csv',
        'cv_fold': 5,
        'cv_val': 0.2,
        'cv_test': 0.2,
        'cv_train': 0.6,
        'cv_equal': True,
        'opt': 'adam',
        'log_data': True,
        'n_classes': 2,
        'lr': 1e-4,
        'reg': 1e-5,
        'drop_out': True,
        'weighted_sample': True,
        'max_epochs': 400,
        'early_stopping': False,
        'model_type': 'clam_mb',
        'bag_loss': 'svm',
        'inst_loss': 'ce',
        'bag_weight': 0.7,
        'B': 2,
        'model_size': 'small',
        'no_inst_cluster': False,
        'subtyping': False,
        'log_data': True,
        'testing': False
       }, index = [0])

In [21]:
from models.model_clam import CLAM_MB, CLAM_SB
from utils.utils import *
from utils.file_utils import save_pkl, load_pkl




def initiate_model(model_args, ckpt_path):
    print('Init Model')    
    model_dict = {
        "dropout": model_args['drop_out'].item(),
        'n_classes': model_args['n_classes'].item(), # model_args.n_classes,
        "k_sample": model_args['B'].item()
    }
    if model_args['model_size'].item() is not None and model_args['model_type'].item() in ['clam_sb', 'clam_mb']:
        model_dict.update({"size_arg": model_args['model_size'].item()})
    if model_args['model_type'].item() =='clam_sb':
        model = CLAM_SB(**model_dict)
    elif model_args['model_type'].item() =='clam_mb':
        model = CLAM_MB(**model_dict)
    else: # args.model_type == 'mil'
        if model_args['n_classes'].item() > 2:
            model = MIL_fc_mc(**model_dict)
        else:
            model = MIL_fc(**model_dict)
    print_network(model)
    ckpt = torch.load(ckpt_path)
    #print_network(ckpt)
    ckpt_clean = {}
    for key in ckpt.keys():
        if 'instance_loss_fn' in key:
            continue
        ckpt_clean.update({key.replace('.module', ''):ckpt[key]})
    model.load_state_dict(ckpt_clean, strict=True)
    model.relocate()
    model.eval()
    return model

# %%
def infer_single_slide(model, features, k=1, instance_eval = False, label_tensor = None):
    features = features.to(device)
    with torch.no_grad():
        if isinstance(model, (CLAM_SB,)):
            #model_results_dict = model(features)
            logits, Y_prob, Y_hat, A, results_dict = model(features, instance_eval = instance_eval)
            print(results_dict)
            Y_hat = Y_hat.item()
            if isinstance(model, (CLAM_MB,)):
                A = A[Y_hat]
            A = A.view(-1, 1).cpu().numpy()
        else:
            raise NotImplementedError
        print('Y_prob: {}', ["{:.4f}".format(p) for p in Y_prob.cpu().flatten()])
        probs, ids = torch.topk(Y_prob, k)
        probs = probs[-1].cpu().numpy()
        ids = ids[-1].cpu().numpy()
        #preds_str = np.array([reverse_label_dict[idx] for idx in ids])
    return ids, probs, A, results_dict


# %%

# %% [markdown]
## Line by Line

# %%
def extract_attention_scores(args, cell_barcode_path, ckpt_path, feature_path, meta_path):
    ###
    cell_barcode_dict = load_pkl(cell_barcode_path)
    metadata = pd.read_csv(meta_path)
    files = os.listdir(feature_path)
    cell_weights = torch.Tensor()
    ###
    sample_list = []
    barcode_list = []
    results_dict_list = []
    ###
    for f in files:
        features = torch.load(os.path.join(feature_path, f))
        model = initiate_model(args, ckpt_path)
        sample_id = f.split('.')[0]
        if sample_id not in str( cell_barcode_dict.keys() ):
            continue
        cell_barcodes = cell_barcode_dict[ int( sample_id )]
        #label = metadata[ metadata['slide_id'] == int( sample_id )].label.values[0]
        #label_dict = {'CTR':0, 'CASE':1 }
        #class_labels = list(label_dict.keys())
        #class_encodings = list(label_dict.values())
        #label_tensor = torch.tensor([label_dict[label]])
        #reverse_label_dict = {class_encodings[i]: class_labels[i] for i in range(len(class_labels))} 
        ids, probs, A, results_dict  = infer_single_slide(model, features, k=1,
                                                        instance_eval = False, label_tensor= None)
        weights = F.softmax(torch.Tensor(A), dim = 0).view(1, -1)
        cell_weights = torch.cat((cell_weights, weights), dim = 1)
        sample_list.extend(np.repeat(sample_id, features.shape[0]))
        barcode_list.extend(cell_barcodes)
        results_dict_list.append(results_dict)
    return cell_weights, sample_list, barcode_list, results_dict_list

# %% [markdown]
## Main

# %%
# args.results_dir = '{}/{}_{}_{}'.format(
# 	args.dir_result,
# 	int( 100 * args.cv_val ), int( 100 * args.cv_test ), args.cv_fold
# )
###
# cell_barcode_path = "{}/insts_in_bags.pkl".format( args.dir_oniput )
# ckpt_path = "{}/s_0_checkpoint.pt".format( args.results_dir )
# feature_path = "{}/bags".format( args.dir_oniput )
# meta_path = "{}/bag.csv".format( args.dir_oniput )

cell_barcode_path = args['cell_barcode_path'].item()
ckpt_path = args['ckpt_path'].item()
feature_path = args['feature_path'].item()
meta_path = args['meta_path'].item()

#%%
# args.ckpt_path = ckpt_path # os.path.join(ckpt_path, f, 's_0_checkpoint.pt')
# args.k_sample = args.B #int(f.split('_B_')[1].split('_bag')[0])
cell_weights, sample_list, barcode_list, results_dict_list = extract_attention_scores( args,
                                                                        cell_barcode_path, #args.barcode_path,
                                                                        ckpt_path, #args.ckpt_path,
                                                                        feature_path, #args.feature_path, 
                                                                        meta_path) #args.meta_path 

# %%
DF = pd.DataFrame( {'bag': sample_list, 'inst': barcode_list, 'weight': cell_weights[0,:]} )
DF

# %%
path = "/Users/chitrakbanerjee/Desktop/omics/tabular-clam/Commingle/weights_oob.csv"
DF.to_csv(path, index = False)

# %%

# %% [markdown]
## EoF


Init Model
CLAM_MB(
  (attention_net): Sequential(
    (0): Linear(in_features=315, out_features=100, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
    (3): Attn_Net_Gated(
      (attention_a): Sequential(
        (0): Linear(in_features=100, out_features=50, bias=True)
        (1): Tanh()
        (2): Dropout(p=0.25, inplace=False)
      )
      (attention_b): Sequential(
        (0): Linear(in_features=100, out_features=50, bias=True)
        (1): Sigmoid()
        (2): Dropout(p=0.25, inplace=False)
      )
      (attention_c): Linear(in_features=50, out_features=2, bias=True)
    )
  )
  (classifiers): ModuleList(
    (0-1): 2 x Linear(in_features=100, out_features=1, bias=True)
  )
  (instance_classifiers): ModuleList(
    (0-1): 2 x Linear(in_features=100, out_features=2, bias=True)
  )
  (instance_loss_fn): CrossEntropyLoss()
)
Total number of parameters: 42408
Total number of trainable parameters: 42408
{}
Y_prob: {} ['0.0000', '1.0000']
Init Model
CLAM_MB(

In [22]:
unlabeled = pd.read_csv('weights_oob.csv')

In [23]:
unlabeled.head()

,bag,inst,weight
0,19,V29_S_4_001_0.jpeg,0.050351
1,19,V29_S_4_001_1.jpeg,0.050351
2,19,V29_S_4_001_2.jpeg,0.029278
3,19,V29_S_4_001_3.jpeg,0.050778
4,19,V29_S_4_001_4.jpeg,0.030027


In [53]:
unlabeled.head()

,Unnamed: 0,bag,inst,weight
0,0,19,V29_S_4_001_0.jpeg,0.048492
1,1,19,V29_S_4_001_1.jpeg,0.052550
2,2,19,V29_S_4_001_2.jpeg,0.073785
3,3,19,V29_S_4_001_3.jpeg,0.088019
4,4,19,V29_S_4_001_4.jpeg,0.046873


In [35]:
unlabeled.groupby('bag')['weight'].sum()

bag
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9     1.0
10    1.0
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    1.0
22    1.0
23    1.0
24    1.0
25    1.0
26    1.0
27    1.0
28    1.0
29    1.0
30    1.0
Name: weight, dtype: float64

In [37]:
x = torch.load('/Users/chitrakbanerjee/Desktop/omics/new_dev/bags_oob/30.pt')

In [38]:
x.shape

torch.Size([22, 316])

In [39]:
x

tensor([[8.3765e-01, 1.8746e-01, 3.3689e-02,  ..., 3.6231e+02, 3.9483e+02,
         3.0000e+01],
        [7.9006e-01, 2.5811e-01, 6.0353e-02,  ..., 3.4161e+02, 3.7819e+02,
         3.0000e+01],
        [7.8716e-01, 2.4623e-01, 7.3277e-02,  ..., 3.9955e+02, 4.2055e+02,
         3.0000e+01],
        ...,
        [7.4368e-01, 3.1099e-01, 7.3491e-02,  ..., 2.5052e+02, 2.6366e+02,
         3.0000e+01],
        [7.4427e-01, 3.4818e-01, 3.6882e-02,  ..., 2.6038e+02, 2.8487e+02,
         3.0000e+01],
        [7.1670e-01, 3.5653e-01, 9.2368e-02,  ..., 3.1335e+02, 3.3705e+02,
         3.0000e+01]])

In [24]:
load_pkl('insts_in_bags.pkl')

{1: array(['10_S_3_0.jpeg', '10_S_3_1.jpeg', '10_S_3_2.jpeg', '10_S_3_3.jpeg',
        '10_S_3_0.jpeg', '10_S_3_1.jpeg', '10_S_3_2.jpeg'], dtype=object),
 2: array(['11_S_3_0.jpeg', '11_S_3_0.jpeg', '11_S_3_1.jpeg', '11_S_3_2.jpeg',
        '11_S_3_3.jpeg', '11_S_3_4.jpeg', '11_S_3_5.jpeg', '11_S_3_6.jpeg',
        '11_S_3_7.jpeg'], dtype=object),
 3: array(['13_S_170019134-3_0.jpeg', '13_S_170019134-3_1.jpeg',
        '13_S_170019134-3_10.jpeg', '13_S_170019134-3_11.jpeg',
        '13_S_170019134-3_12.jpeg', '13_S_170019134-3_13.jpeg',
        '13_S_170019134-3_14.jpeg', '13_S_170019134-3_15.jpeg',
        '13_S_170019134-3_16.jpeg', '13_S_170019134-3_17.jpeg',
        '13_S_170019134-3_18.jpeg', '13_S_170019134-3_19.jpeg',
        '13_S_170019134-3_2.jpeg', '13_S_170019134-3_20.jpeg',
        '13_S_170019134-3_21.jpeg', '13_S_170019134-3_22.jpeg',
        '13_S_170019134-3_23.jpeg', '13_S_170019134-3_3.jpeg',
        '13_S_170019134-3_4.jpeg', '13_S_170019134-3_5.jpeg',
        '13_S

In [1]:
from topk.svm import SmoothTop1SVM

In [31]:
output = torch.randn(10, 1)

In [32]:
output

tensor([[ 0.5221],
        [ 0.0993],
        [ 1.2200],
        [ 0.2630],
        [ 0.0533],
        [-0.0296],
        [ 0.1782],
        [-0.8404],
        [-0.7691],
        [ 1.0615]])

In [33]:
output.view(1, -1)

tensor([[ 0.5221,  0.0993,  1.2200,  0.2630,  0.0533, -0.0296,  0.1782, -0.8404,
         -0.7691,  1.0615]])

In [34]:
maxk = 1
torch.topk(output, 1)

torch.return_types.topk(
values=tensor([[ 0.5221],
        [ 0.0993],
        [ 1.2200],
        [ 0.2630],
        [ 0.0533],
        [-0.0296],
        [ 0.1782],
        [-0.8404],
        [-0.7691],
        [ 1.0615]]),
indices=tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]]))

In [35]:
len(output.view(1, -1).shape)

2

In [23]:
maxk = 2
torch.topk(output, 1)[1][-1]

tensor([1])

In [16]:
args['cv_val'].item()

0.2

In [10]:
csv_path = os.path.join( '/Users/chitrakbanerjee/Desktop/omics/new_dev/split/', '{}_{}_{}_{}'.format(int( 100 * cv_val ), 
        int( 100 * cv_test ), cv_fold, f) + '.csv' )

In [11]:
d = dataset.return_splits(from_id=False, csv_path=csv_path, val_frac=0.2, test_frac=0.2 )

In [12]:
train_split = dataset.get_split_from_df(pd.read_csv(csv_path), 'train')

In [13]:
train_split[0]

(tensor([[7.6374e-01, 2.9191e-01, 4.6722e-02,  ..., 2.5534e+02, 2.6834e+02,
          2.8834e+02],
         [7.6065e-01, 3.1171e-01, 4.9396e-02,  ..., 4.6671e+02, 4.8443e+02,
          5.0843e+02],
         [7.1368e-01, 3.7529e-01, 6.6800e-02,  ..., 1.7643e+02, 1.8714e+02,
          2.0361e+02],
         ...,
         [7.5604e-01, 2.9954e-01, 4.7077e-02,  ..., 9.6748e+01, 1.0521e+02,
          1.1272e+02],
         [7.6887e-01, 3.0632e-01, 2.8374e-02,  ..., 1.6174e+02, 1.6955e+02,
          1.7655e+02],
         [7.6065e-01, 3.3915e-01, 2.0174e-02,  ..., 1.5948e+02, 1.6918e+02,
          1.7818e+02]]),
 0)

In [36]:
test_dataset[0][0]

tensor([[7.5540e-01, 2.9897e-01, 5.7978e-02,  ..., 4.0787e+02, 4.2369e+02,
         4.4192e+02],
        [7.3131e-01, 3.3108e-01, 7.8102e-02,  ..., 4.1476e+02, 4.3185e+02,
         4.5124e+02],
        [7.4528e-01, 3.3066e-01, 5.4547e-02,  ..., 4.1105e+02, 4.2760e+02,
         4.4498e+02],
        ...,
        [7.6000e-01, 3.0634e-01, 5.9379e-02,  ..., 3.5635e+02, 3.6736e+02,
         3.8192e+02],
        [7.7293e-01, 2.9386e-01, 3.7334e-02,  ..., 3.5137e+02, 3.6224e+02,
         3.7637e+02],
        [7.5152e-01, 3.4991e-01, 1.1812e-02,  ..., 1.3392e+02, 1.5302e+02,
         2.0238e+02]])